In [ ]:
# Import libraries
import pandas as pd
import re
import unicodedata
import demoji

# Download emoji data (run once)
demoji.download_codes()

C:\Users\rhzain\AppData\Local\Temp\ipykernel_33724\1846252205.py:8: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [4]:
df = pd.read_csv('data_filtered.csv')
print(f"Total komentar: {len(df)}")
print("\nContoh data awal:")
df.head()

Total komentar: 778

Contoh data awal:


,Video_ID,Teks_Komentar
0,MIo4tGN11j0,"Sempat mikir mau pindah ke negara sebelah, nge..."
1,MIo4tGN11j0,"Kalo kabur mau kemana ke Singapur ,emang di Si..."
2,MIo4tGN11j0,Klo sudah gelap susah terangnya lebih baik bubar
3,MIo4tGN11j0,Pengen kabur tapi gak punya uang😭
4,MIo4tGN11j0,Kalau mau pindah ke luar negeri ya silahkan sa...


## 2. Preprocessing Teks

Melakukan preprocessing data dengan 5 tahap terpisah yang modular

In [5]:
import pandas as pd
import demoji
from nltk.corpus import stopwords

# --- 1. Inisialisasi Alat (Sekali saja) ---
print("="*60)
print("INISIALISASI PREPROCESSING TOOLS")
print("="*60)

# Impor Stemmer dari Sastrawi
try:
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    print("✅ Sastrawi Stemmer berhasil dimuat")
    factory = StemmerFactory()
    stemmer_sastrawi = factory.create_stemmer()
except ImportError:
    print("❌ ERROR: Library Sastrawi tidak ditemukan.")
    print("   Silakan instal dengan: pip install Sastrawi")
    raise

# Import indoNLP untuk preprocessing slang dan elongation
try:
    from indonlp.preprocessing import replace_slang, replace_word_elongation
    print("✅ indoNLP preprocessing berhasil dimuat")
except ImportError:
    print("⚠️  WARNING: Library indoNLP tidak ditemukan.")
    print("   Silakan instal dengan: pip install indonlp")
    print("   Preprocessing akan tetap berjalan tanpa indoNLP.")
    replace_slang = lambda x: x
    replace_word_elongation = lambda x: x

# --- 2. Buat Daftar KEYWORD untuk Filter (dari Cell 4) ---
KEYWORD = [
    'kabur', 'pindah negara', 'leave indo', 'pajak', 'gaji', 'umr', 
    'biaya hidup', 'korupsi', 'sandwich', 'luar negeri', 
    'paspor', 'warga negara', 'singapur', 'singapore', 'australia', 'aussie', 'jepang', 'eropa', 'wni',
    'capek', 'lelah', 'pemerintah', 'mending', 'percuma', 'beban', 
    'suram', 'males', 'ga jelas', 'ga ada harapan', 'nyicil', 'susah',
    'setuju', 'sulit', 'stres', 'politik', 'birokrasi', 'konoha'
]

# Buat set untuk pencarian cepat saat stemming
KEYWORD_SET = set()
for keyword in KEYWORD:
    # Pecah multi-word keywords jadi single word
    KEYWORD_SET.update(keyword.lower().split())

print(f"✅ KEYWORD dimuat: {len(KEYWORD)} frasa → {len(KEYWORD_SET)} kata unik")

# --- 3. Buat Daftar Stopwords Gabungan ---
stop_words_indo = set(stopwords.words('indonesian'))
stop_words_eng = set(stopwords.words('english'))

# Stopwords gaul dan kata rojak (bahasa Inggris umum)
custom_stopwords_gaul = {
    'yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 'kalo', 'amp', 'biar', 'bkn', 'na', 
    'nya', 'nih', 'sih', 'si', 'tau', 'tuh', 'utk', 'ya', 'gaes',
    'bang', 'bro', 'sob', 'gw', 'gua', 'lu', 'lo', 'wkwk', 'haha', 'wkwkwk', 
    'amin', 'amiin', 'aamiin', 'yuk', 'dong', 'deh', 'kok',
    # Kata rojak (bahasa Inggris umum yang tidak bisa di-stem)
    'government', 'tax', 'salary', 'system', 'netizen', 'the', 'and', 'or', 'in', 'of', 'to', 'is', 'for'
}

# --- 4. Daftar Kata NEGASI yang TIDAK BOLEH dihapus ---
NEGASI_WORDS = {
    'tidak', 'bukan', 'jangan', 'ga', 'gak', 'enggak', 'nggak', 'ndak', 'engga', 'belum', 'tanpa'
}

# Gabungkan semua stopwords, KECUALI kata negasi dan keyword penting
stop_words_final = (stop_words_indo.union(stop_words_eng, custom_stopwords_gaul)) - NEGASI_WORDS - KEYWORD_SET

print("✅ Stopwords berhasil dimuat:")
print(f"   - Indonesia       : {len(stop_words_indo)} kata")
print(f"   - Inggris         : {len(stop_words_eng)} kata")
print(f"   - Gaul + Rojak    : {len(custom_stopwords_gaul)} kata")
print(f"   - TOTAL           : {len(stop_words_final)} kata (setelah exclude negasi & keyword)")
print(f"   - Negasi (PROTECT): {NEGASI_WORDS}")
print(f"   - Keyword (PROTECT): {len(KEYWORD_SET)} kata")

print("\n" + "="*60)


# --- 5. Definisi Fungsi Preprocessing Pipeline ---

def tahap1_cleaning(teks):
    """
    Tahap 2: Membersihkan teks mentah dari URL, HTML, emoji,
    normalisasi slang, elongasi, tanda baca, dan angka.
    
    DENGAN indoNLP untuk slang dan elongation!
    """
    if pd.isna(teks):
        return ""
    
    teks = str(teks)
    
    # 1. Hapus URL
    teks = re.sub(r'http\S+|www\.\S+', '', teks)
    
    # 2. Hapus HTML tags
    teks = re.sub(r'<.*?>', '', teks)
    
    # 3. Hapus SEMUA emoji (tanpa konversi ke tag)
    teks = demoji.replace(teks, '')
    
    # 4. Normalisasi Unicode (font aneh)
    teks = unicodedata.normalize('NFKD', teks)
    
    # 5. Lowercase
    teks = teks.lower()
    
    # 6. Ganti slang menggunakan indoNLP
    teks = replace_slang(teks)
    
    # 7. Ganti elongasi (kata berulang) menggunakan indoNLP
    teks = replace_word_elongation(teks)
    
    # 8. Hapus tanda baca dan angka (sisakan huruf dan spasi)
    teks = re.sub(r'[^a-z\s]', ' ', teks)
    
    # 9. Hapus spasi berlebih
    teks = re.sub(r'\s+', ' ', teks).strip()
    
    return teks


def tahap2_tokenisasi(teks):
    """
    Tahap 3a: Tokenisasi - Memecah teks menjadi list kata-kata.
    """
    if not teks or pd.isna(teks):
        return []
    
    # Pecah berdasarkan spasi
    tokens = teks.split()
    
    # Filter kata yang terlalu pendek (< 3 karakter), KECUALI kata negasi pendek
    tokens = [kata for kata in tokens if len(kata) >= 3 or kata in {'ga', 'gak'}]
    
    return tokens


def tahap2_5_negation_tagging(tokens):
    """
    Tahap 3b: Negation Tagging - Menggabungkan kata negasi dengan kata berikutnya.
    
    Implementasi negation tagging untuk analisis sentimen yang lebih baik.
    Contoh: ['gaji', 'ga', 'naik'] -> ['gaji', 'TIDAK_naik']
    """
    if not tokens or len(tokens) == 0:
        return []
    
    tokens_tagged = []
    i = 0
    
    while i < len(tokens):
        current_token = tokens[i]
        
        # Jika kata ini adalah negasi DAN ada kata setelahnya
        if current_token in NEGASI_WORDS and i + 1 < len(tokens):
            next_token = tokens[i + 1]
            # Gabungkan negasi dengan kata berikutnya
            tagged_token = f"TIDAK_{next_token}"
            tokens_tagged.append(tagged_token)
            # Skip kata berikutnya karena sudah digabung
            i += 2
        else:
            # Kata biasa, tambahkan seperti biasa
            tokens_tagged.append(current_token)
            i += 1
    
    return tokens_tagged


def tahap4_stemming(tokens):
    """
    Tahap 4: Stemming - Mengubah kata ke bentuk dasar.
    
    PENTING: Skip stemming untuk:
    - Kata yang ada di KEYWORD_SET (kata kunci penting topik)
    - Tag TIDAK_ (negation tagging)
    """
    if not tokens:
        return []
    
    tokens_stemmed = []
    for kata in tokens:
        # Jika kata adalah keyword penting, JANGAN di-stem
        if kata in KEYWORD_SET:
            tokens_stemmed.append(kata)
        # Jika kata mengandung tag TIDAK_, stem hanya bagian kata dasarnya
        elif kata.startswith('TIDAK_'):
            # Ambil kata setelah TIDAK_
            kata_dasar = kata.replace('TIDAK_', '')
            # Cek apakah kata dasar adalah keyword
            if kata_dasar in KEYWORD_SET:
                # Jangan stem, pertahankan dengan tag TIDAK_
                tokens_stemmed.append(kata)
            else:
                # Stem kata dasarnya
                kata_stemmed = stemmer_sastrawi.stem(kata_dasar)
                # Gabung kembali dengan tag TIDAK_
                tokens_stemmed.append(f'TIDAK_{kata_stemmed}')
        else:
            # Kata biasa, stem seperti biasa
            tokens_stemmed.append(stemmer_sastrawi.stem(kata))
    
    return tokens_stemmed


def tahap3_stopword_removal(tokens):
    """
    Tahap 5: Hapus Stopwords dari list tokens.
    
    Dilakukan SETELAH stemming.
    JANGAN hapus: tag TIDAK_ dan keyword penting.
    """
    if not tokens:
        return []
    
    tokens_bersih = []
    for kata in tokens:
        # Jangan hapus kata yang mengandung tag TIDAK_
        if kata.startswith('TIDAK_'):
            tokens_bersih.append(kata)
        # Jangan hapus keyword penting
        elif kata in KEYWORD_SET:
            tokens_bersih.append(kata)
        # Hapus stopwords biasa
        elif kata not in stop_words_final:
            tokens_bersih.append(kata)
    
    return tokens_bersih


def tahap5_gabung_kembali(tokens):
    """
    Tahap 6: Gabungkan tokens menjadi teks final.
    """
    if not tokens:
        return ""
    
    return " ".join(tokens)

print("✅ Semua fungsi preprocessing pipeline sudah siap!")
print("\n📋 URUTAN PIPELINE:")
print("   Tahap 1: Filter Konteks & Noise")
print("   Tahap 2: Cleaning (URL, HTML, Hapus Emoji, Slang, Elongasi, Tanda Baca)")
print("   Tahap 3: Tokenisasi + Negation Tagging")
print("   Tahap 4: Stemming (skip keyword & tag)")
print("   Tahap 5: Stopword Removal (preserve negasi & keyword)")
print("   Tahap 6: Gabung Kembali")
print("="*60)


INISIALISASI PREPROCESSING TOOLS
✅ Sastrawi Stemmer berhasil dimuat
⚠️  WARNING: Library indoNLP tidak ditemukan.
   Silakan instal dengan: pip install indonlp
   Preprocessing akan tetap berjalan tanpa indoNLP.
✅ KEYWORD dimuat: 37 frasa → 43 kata unik
✅ Stopwords berhasil dimuat:
   - Indonesia       : 757 kata
   - Inggris         : 198 kata
   - Gaul + Rojak    : 51 kata
   - TOTAL           : 985 kata (setelah exclude negasi & keyword)
   - Negasi (PROTECT): {'gak', 'belum', 'ndak', 'nggak', 'jangan', 'tanpa', 'engga', 'ga', 'bukan', 'tidak', 'enggak'}
   - Keyword (PROTECT): 43 kata

✅ Semua fungsi preprocessing pipeline sudah siap!

📋 URUTAN PIPELINE:
   Tahap 1: Filter Konteks & Noise
   Tahap 2: Cleaning (URL, HTML, Hapus Emoji, Slang, Elongasi, Tanda Baca)
   Tahap 3: Tokenisasi + Negation Tagging
   Tahap 4: Stemming (skip keyword & tag)
   Tahap 5: Stopword Removal (preserve negasi & keyword)
   Tahap 6: Gabung Kembali


### Tahap 1: Cleaning

Membersihkan teks dari URL, HTML, emoji, tanda baca, dan angka

In [6]:
print("="*60)
print("📝 TAHAP 2/6: CLEANING")
print("="*60)
print("Membersihkan URL, HTML, emoji→tag, slang, elongasi, tanda baca...")

df['teks_tahap1'] = df['Teks_Komentar'].apply(tahap1_cleaning)

print("✅ Selesai!")
print(f"\nContoh hasil cleaning:")
print(f"ASLI  : {df['Teks_Komentar'].iloc[0][:80]}...")
print(f"CLEAN : {df['teks_tahap1'].iloc[0][:80]}...")
print("="*60)


📝 TAHAP 2/6: CLEANING
Membersihkan URL, HTML, emoji→tag, slang, elongasi, tanda baca...
✅ Selesai!

Contoh hasil cleaning:
ASLI  : Sempat mikir mau pindah ke negara sebelah, ngeapply citizenship. Tapi yah, aku c...
CLEAN : sempat mikir mau pindah ke negara sebelah ngeapply citizenship tapi yah aku cint...
✅ Selesai!

Contoh hasil cleaning:
ASLI  : Sempat mikir mau pindah ke negara sebelah, ngeapply citizenship. Tapi yah, aku c...
CLEAN : sempat mikir mau pindah ke negara sebelah ngeapply citizenship tapi yah aku cint...


### Tahap 3: Tokenisasi & Negation Tagging

Memecah teks menjadi kata-kata dan menggabungkan kata negasi dengan kata berikutnya

In [7]:
print("="*60)
print("📝 TAHAP 3/6: TOKENISASI & NEGATION TAGGING")
print("="*60)
print("Memecah teks menjadi kata-kata...")

# Tahap 3a: Tokenisasi
df['tokens_tahap2'] = df['teks_tahap1'].apply(tahap2_tokenisasi)

print("✅ Tokenisasi selesai!")
print(f"\nContoh hasil tokenisasi:")
tokens_contoh = df['tokens_tahap2'].iloc[0]
print(f"TEKS   : {df['teks_tahap1'].iloc[0][:60]}...")
print(f"TOKENS : {tokens_contoh[:8]}...")
print(f"JUMLAH : {len(tokens_contoh)} kata")

# Tahap 3b: Negation Tagging
print("\nMenggabungkan kata negasi dengan kata berikutnya...")
df['tokens_tahap2_5'] = df['tokens_tahap2'].apply(tahap2_5_negation_tagging)

print("✅ Negation tagging selesai!")
print(f"\nContoh hasil negation tagging:")
tokens_before = df['tokens_tahap2'].iloc[0]
tokens_after = df['tokens_tahap2_5'].iloc[0]
print(f"SEBELUM : {tokens_before[:8]}...")
print(f"SESUDAH : {tokens_after[:8]}...")
print(f"\n💡 Contoh: 'gaji ga naik' → 'gaji TIDAK_naik'")
print("="*60)


📝 TAHAP 3/6: TOKENISASI & NEGATION TAGGING
Memecah teks menjadi kata-kata...
✅ Tokenisasi selesai!

Contoh hasil tokenisasi:
TEKS   : sempat mikir mau pindah ke negara sebelah ngeapply citizensh...
TOKENS : ['sempat', 'mikir', 'mau', 'pindah', 'negara', 'sebelah', 'ngeapply', 'citizenship']...
JUMLAH : 21 kata

Menggabungkan kata negasi dengan kata berikutnya...
✅ Negation tagging selesai!

Contoh hasil negation tagging:
SEBELUM : ['sempat', 'mikir', 'mau', 'pindah', 'negara', 'sebelah', 'ngeapply', 'citizenship']...
SESUDAH : ['sempat', 'mikir', 'mau', 'pindah', 'negara', 'sebelah', 'ngeapply', 'citizenship']...

💡 Contoh: 'gaji ga naik' → 'gaji TIDAK_naik'


### Tahap 4: Stemming

Mengubah kata ke bentuk dasar (SKIP keyword penting dan tag)

In [9]:
print("="*60)
print("📝 TAHAP 4/6: STEMMING")
print("="*60)
print("Mengubah kata ke bentuk dasar...")
print("⚠️  Proses ini lambat, harap sabar...")
print("💡 Keyword penting & tag (TIDAK_, _EMOJI_) akan di-skip")

df['tokens_tahap3'] = df['tokens_tahap2_5'].apply(tahap4_stemming)

print("✅ Selesai!")
print(f"\nContoh hasil stemming:")
tokens_before = df['tokens_tahap2_5'].iloc[0]
tokens_after = df['tokens_tahap3'].iloc[0]
print(f"SEBELUM : {tokens_before[:6]}")
print(f"SESUDAH : {tokens_after[:6]}")
print("="*60)


📝 TAHAP 4/6: STEMMING
Mengubah kata ke bentuk dasar...
⚠️  Proses ini lambat, harap sabar...
💡 Keyword penting & tag (TIDAK_, _EMOJI_) akan di-skip
✅ Selesai!

Contoh hasil stemming:
SEBELUM : ['sempat', 'mikir', 'mau', 'pindah', 'negara', 'sebelah']
SESUDAH : ['sempat', 'mikir', 'mau', 'pindah', 'negara', 'belah']
✅ Selesai!

Contoh hasil stemming:
SEBELUM : ['sempat', 'mikir', 'mau', 'pindah', 'negara', 'sebelah']
SESUDAH : ['sempat', 'mikir', 'mau', 'pindah', 'negara', 'belah']


### Tahap 5: Stopword Removal

Menghapus stopwords SETELAH stemming (preserve negasi, emoji, dan keyword)

In [10]:
print("="*60)
print("📝 TAHAP 5/6: STOPWORD REMOVAL")
print("="*60)
print("Menghapus stopwords (preserve: negasi, emoji tag, keyword)...")

df['tokens_tahap4'] = df['tokens_tahap3'].apply(tahap3_stopword_removal)

print("✅ Selesai!")
print(f"\nContoh hasil stopword removal:")
tokens_before = df['tokens_tahap3'].iloc[0]
tokens_after = df['tokens_tahap4'].iloc[0]
print(f"SEBELUM : {tokens_before[:8]}...")
print(f"SESUDAH : {tokens_after[:8]}...")
print(f"JUMLAH  : {len(tokens_before)} kata → {len(tokens_after)} kata")
print(f"REDUKSI : {len(tokens_before) - len(tokens_after)} kata dihapus")
print("="*60)


📝 TAHAP 5/6: STOPWORD REMOVAL
Menghapus stopwords (preserve: negasi, emoji tag, keyword)...
✅ Selesai!

Contoh hasil stopword removal:
SEBELUM : ['sempat', 'mikir', 'mau', 'pindah', 'negara', 'belah', 'ngeapply', 'citizenship']...
SESUDAH : ['mikir', 'pindah', 'negara', 'belah', 'ngeapply', 'citizenship', 'yah', 'cinta']...
JUMLAH  : 21 kata → 14 kata
REDUKSI : 7 kata dihapus


### Tahap 6: Gabung Kembali & Simpan

Menggabungkan tokens menjadi teks final dan menyimpan hasil

In [11]:
print("="*60)
print("📝 TAHAP 6/6: GABUNG KEMBALI & SIMPAN")
print("="*60)
print("Menggabungkan tokens menjadi teks final...")

df['teks_final'] = df['tokens_tahap4'].apply(tahap5_gabung_kembali)

# Buang baris dengan teks kosong setelah preprocessing
df_final = df[df['teks_final'].str.strip() != ''].copy()

print("✅ Selesai!")
print(f"\n📊 Ringkasan:")
print(f"   Total komentar valid     : {len(df_final):,}")
print(f"   Komentar kosong terbuang : {len(df) - len(df_final):,}")

# Simpan hasil akhir
output_file = 'data_preprocessed.csv'
df_final[['Teks_Komentar', 'teks_final']].to_csv(output_file, index=False, encoding='utf-8')
print(f"\n💾 Data berhasil disimpan ke '{output_file}'")

# Tampilkan contoh hasil
print(f"\n📋 Contoh hasil akhir:")
for i in range(min(3, len(df_final))):
    print(f"\n   [{i+1}] ASLI  : {df_final['Teks_Komentar'].iloc[i][:60]}...")
    print(f"       FINAL : {df_final['teks_final'].iloc[i][:60]}...")

print("\n" + "="*60)
print("🎉 PREPROCESSING SELESAI!")
print("="*60)
print("\n📌 PIPELINE YANG DITERAPKAN:")
print("   ✅ Tahap 1: Filter Konteks & Noise (Strategi LONGGAR)")
print("   ✅ Tahap 2: Cleaning (URL, HTML, Hapus Emoji, Slang, Elongasi)")
print("   ✅ Tahap 3: Tokenisasi + Negation Tagging")
print("   ✅ Tahap 4: Stemming (skip keyword & tag)")
print("   ✅ Tahap 5: Stopword Removal (preserve negasi & keyword)")
print("   ✅ Tahap 6: Gabung Kembali")
print("\n📌 FITUR PERBAIKAN:")
print("   ✅ Urutan BENAR: Stemming → Stopword Removal")
print("   ✅ Kata negasi DIPERTAHANKAN & di-tag (ga naik → TIDAK_naik)")
print("   ✅ Emoji → DIHAPUS (tidak digunakan sebagai fitur)")
print("   ✅ Kata rojak (government, tax, dll) → dihapus")
print("   ✅ Keyword penting (pajak, gaji, dll) → TIDAK di-stem")
print("   ✅ Slang & elongasi → dinormalisasi (indoNLP)")
print("\n📝 UNTUK MODELLING:")
print("   ⚠️  Gunakan ngram_range=(1,2) di TF-IDF untuk bigram!")
print("="*60)


📝 TAHAP 6/6: GABUNG KEMBALI & SIMPAN
Menggabungkan tokens menjadi teks final...
✅ Selesai!

📊 Ringkasan:
   Total komentar valid     : 778
   Komentar kosong terbuang : 0

💾 Data berhasil disimpan ke 'data_preprocessed.csv'

📋 Contoh hasil akhir:

   [1] ASLI  : Sempat mikir mau pindah ke negara sebelah, ngeapply citizens...
       FINAL : mikir pindah negara belah ngeapply citizenship yah cinta neg...

   [2] ASLI  : Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal...
       FINAL : kabur singapur emang singapur tinggal rakyat singapur aja TI...

   [3] ASLI  : Klo sudah gelap susah terangnya lebih baik bubar...
       FINAL : gelap susah terang bubar...

🎉 PREPROCESSING SELESAI!

📌 PIPELINE YANG DITERAPKAN:
   ✅ Tahap 1: Filter Konteks & Noise (Strategi LONGGAR)
   ✅ Tahap 2: Cleaning (URL, HTML, Hapus Emoji, Slang, Elongasi)
   ✅ Tahap 3: Tokenisasi + Negation Tagging
   ✅ Tahap 4: Stemming (skip keyword & tag)
   ✅ Tahap 5: Stopword Removal (preserve negasi & keyword)
 

### Tahap 5: Gabung Kembali & Simpan

Menggabungkan tokens menjadi teks final dan menyimpan hasil

In [ ]:
# Lihat detail preprocessing untuk 3 contoh pertama
print("="*80)
print("DETAIL HASIL SETIAP TAHAP PREPROCESSING")
print("="*80)

for i in range(min(3, len(df_final))):
    print(f"\n{'='*80}")
    print(f"CONTOH {i+1}")
    print(f"{'='*80}")
    print(f"\n0. ASLI:")
    print(f"   {df_final['Teks_Komentar'].iloc[i][:100]}...")
    
    print(f"\n1. CLEANING (Tahap 2):")
    print(f"   {df_final['teks_tahap1'].iloc[i][:100]}...")
    
    print(f"\n2. TOKENISASI (Tahap 3a):")
    tokens_2 = df_final['tokens_tahap2'].iloc[i]
    print(f"   {tokens_2[:10]}...")
    print(f"   (Total: {len(tokens_2)} kata)")
    
    print(f"\n3. NEGATION TAGGING (Tahap 3b):")
    tokens_2_5 = df_final['tokens_tahap2_5'].iloc[i]
    print(f"   {tokens_2_5[:10]}...")
    print(f"   (Total: {len(tokens_2_5)} kata)")
    
    print(f"\n4. STEMMING (Tahap 4):")
    tokens_3 = df_final['tokens_tahap3'].iloc[i]
    print(f"   {tokens_3[:10]}...")
    print(f"   (Total: {len(tokens_3)} kata)")
    
    print(f"\n5. STOPWORD REMOVAL (Tahap 5):")
    tokens_4 = df_final['tokens_tahap4'].iloc[i]
    print(f"   {tokens_4[:10]}...")
    print(f"   (Total: {len(tokens_4)} kata)")
    
    print(f"\n6. FINAL - GABUNG KEMBALI (Tahap 6):")
    print(f"   {df_final['teks_final'].iloc[i][:100]}...")
    print()

print("="*80)
print("\n*** PERHATIKAN ***")
print("   - Kata negasi tetap ada sebagai tag TIDAK_xxx")
print("   - Emoji telah DIHAPUS dari data")
print("   - Keyword penting (pajak, gaji, dll) tidak di-stem")
print("   - Slang sudah dinormalisasi (jika menggunakan indoNLP)")
print("="*80)


DETAIL HASIL SETIAP TAHAP PREPROCESSING

CONTOH 1

0. ASLI:
   Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal dimana rakyat Singapur aja nggak punya ...

1. CLEANING (Tahap 2):
   kalo kabur mau kemana ke singapur emang di singapur tinggal dimana rakyat singapur aja nggak punya r...

2. TOKENISASI (Tahap 3a):
   ['kalo', 'kabur', 'mau', 'kemana', 'singapur', 'emang', 'singapur', 'tinggal', 'dimana', 'rakyat']...
   (Total: 22 kata)

3. NEGATION TAGGING (Tahap 3b):
   ['kalo', 'kabur', 'mau', 'kemana', 'singapur', 'emang', 'singapur', 'tinggal', 'dimana', 'rakyat']...
   (Total: 21 kata)

4. STEMMING (Tahap 4):
   ['kalo', 'kabur', 'mau', 'mana', 'singapur', 'emang', 'singapur', 'tinggal', 'mana', 'rakyat']...
   (Total: 21 kata)

5. STOPWORD REMOVAL (Tahap 5):
   ['kabur', 'singapur', 'emang', 'singapur', 'tinggal', 'rakyat', 'singapur', 'aja', 'TIDAK_punya', 'rumah']...
   (Total: 14 kata)

6. FINAL - GABUNG KEMBALI (Tahap 6):
   kabur singapur emang singapur tinggal rak

## 3. Lihat Detail Hasil Preprocessing (Opsional)

Melihat hasil setiap tahap preprocessing untuk beberapa contoh

In [ ]:
# Hitung statistik lengkap
total_awal = len(df)
total_setelah_filter = len(df)
total_final = len(df_final)
persentase_valid = (total_final / total_awal * 100) if total_awal > 0 else 0

# Hitung rata-rata panjang teks
avg_len_asli = df_final['Teks_Komentar'].str.len().mean()
avg_len_final = df_final['teks_final'].str.len().mean()

# Hitung rata-rata jumlah kata
avg_words_asli = df_final['Teks_Komentar'].str.split().str.len().mean()
avg_words_final = df_final['teks_final'].str.split().str.len().mean()

# Hitung statistik tokens per tahap
avg_tokens_tahap2 = df_final['tokens_tahap2'].apply(len).mean()
avg_tokens_tahap2_5 = df_final['tokens_tahap2_5'].apply(len).mean()
avg_tokens_tahap3 = df_final['tokens_tahap3'].apply(len).mean()
avg_tokens_tahap4 = df_final['tokens_tahap4'].apply(len).mean()

# Hitung berapa banyak negasi yang di-tag
def count_negation_tags(tokens):
    return sum(1 for token in tokens if token.startswith('TIDAK_'))

total_negation_tags = df_final['tokens_tahap4'].apply(count_negation_tags).sum()

# Emoji sudah dihapus, tidak perlu hitung emoji tags

print("=" * 75)
print("📊 STATISTIK PREPROCESSING LENGKAP")
print("=" * 75)

print(f"\n📈 JUMLAH DATA:")
print(f"   {'Data awal (raw)':<35} : {total_awal:>10,} komentar")
print(f"   {'Setelah filter konteks & noise':<35} : {total_setelah_filter:>10,} komentar")
print(f"   {'Data final valid':<35} : {total_final:>10,} komentar")
print(f"   {'Persentase data valid':<35} : {persentase_valid:>10.2f}%")
print(f"   {'Data terbuang':<35} : {total_awal - total_final:>10,} komentar")

print(f"\n📝 RATA-RATA PANJANG TEKS (karakter):")
print(f"   {'Teks asli':<35} : {avg_len_asli:>10.2f}")
print(f"   {'Teks final (preprocessed)':<35} : {avg_len_final:>10.2f}")
print(f"   {'Reduksi':<35} : {((avg_len_asli - avg_len_final) / avg_len_asli * 100):>10.2f}%")

print(f"\n📄 RATA-RATA JUMLAH KATA:")
print(f"   {'Teks asli':<35} : {avg_words_asli:>10.2f} kata")
print(f"   {'Teks final (preprocessed)':<35} : {avg_words_final:>10.2f} kata")
print(f"   {'Reduksi':<35} : {((avg_words_asli - avg_words_final) / avg_words_asli * 100):>10.2f}%")

print(f"\n🔄 RATA-RATA TOKENS PER TAHAP:")
print(f"   {'Setelah Tokenisasi (3a)':<35} : {avg_tokens_tahap2:>10.2f} tokens")
print(f"   {'Setelah Negation Tagging (3b)':<35} : {avg_tokens_tahap2_5:>10.2f} tokens")
print(f"   {'Setelah Stemming (4)':<35} : {avg_tokens_tahap3:>10.2f} tokens")
print(f"   {'Setelah Stopword Removal (5)':<35} : {avg_tokens_tahap4:>10.2f} tokens")
print(f"   {'Reduksi total tokens':<35} : {((avg_tokens_tahap2 - avg_tokens_tahap4) / avg_tokens_tahap2 * 100):>10.2f}%")

print(f"\n🏷️  FITUR KHUSUS:")
print(f"   {'Total TIDAK_xxx tags':<35} : {total_negation_tags:>10,} tag")
print(f"   {'Keyword yang di-protect':<35} : {len(KEYWORD_SET):>10,} kata")
print(f"   {'Kata negasi yang di-protect':<35} : {len(NEGASI_WORDS):>10,} kata")

print(f"   {'Keyword yang di-protect':<35} : {len(KEYWORD_SET):>10,} kata")
print(f"   {'Kata negasi yang di-protect':<35} : {len(NEGASI_WORDS):>10,} kata")

print("\n" + "=" * 75)
print("✅ Preprocessing berhasil!")
print(f"💾 File tersimpan: {output_file}")
print("\n📌 PIPELINE YANG DITERAPKAN:")
print("   1. ✅ Filter Konteks & Noise (Strategi LONGGAR)")
print("   2. ✅ Cleaning (URL, HTML, Emoji→Tag, Slang*, Elongasi*)")
print("   2. ✅ Cleaning (URL, HTML, Hapus Emoji, Slang*, Elongasi*)")
print("   4. ✅ Stemming (skip keyword & tag)")
print("   5. ✅ Stopword Removal (preserve negasi, emoji, keyword)")
print("   5. ✅ Stopword Removal (preserve negasi & keyword)")
print("\n📌 PERBAIKAN UTAMA:")
print("   ✅ Urutan BENAR: Stemming dilakukan SEBELUM Stopword Removal")
print("   ✅ Kata negasi DIPERTAHANKAN dan di-tag dengan kata berikutnya")
print("   ✅ Emoji diubah menjadi tag sentimen sederhana")
print("   ✅ Emoji DIHAPUS dari data (tidak digunakan sebagai fitur)")
print("   ✅ Keyword topik penting TIDAK di-stem untuk preserve makna")
print("   ⭐ Slang & elongasi dinormalisasi (jika indoNLP terinstall)")
print("\n📝 CATATAN UNTUK FASE MODELLING:")
print("   ⚠️  Gunakan ngram_range=(1, 2) saat TF-IDF Vectorization")

📊 STATISTIK PREPROCESSING LENGKAP

📈 JUMLAH DATA:
   Data awal (raw)                     :      2,196 komentar
   Setelah filter konteks & noise      :        641 komentar
   Data final valid                    :        641 komentar
   Persentase data valid               :      29.19%
   Data terbuang                       :      1,555 komentar

📝 RATA-RATA PANJANG TEKS (karakter):
   Teks asli                           :     209.97
   Teks final (preprocessed)           :     117.70
   Reduksi                             :      43.94%

📄 RATA-RATA JUMLAH KATA:
   Teks asli                           :      32.66 kata
   Teks final (preprocessed)           :      17.64 kata
   Reduksi                             :      45.98%

🔄 RATA-RATA TOKENS PER TAHAP:
   Setelah Tokenisasi (3a)             :      29.84 tokens
   Setelah Negation Tagging (3b)       :      29.13 tokens
   Setelah Stemming (4)                :      29.13 tokens
   Setelah Stopword Removal (5)        :      17.64 token